In [1]:
# import numpy
import numpy as np

# module for system control
import os
import argparse

# module for webcam and capturing a webcam screenshot
import cv2
from PIL import Image

# modules for face recognition
import face_recognition
import pickle

In [7]:
# construct the argument parser and parse the arguments
INPUT = '/Users/woolee/mldl_project/github/crop_faces_from_image/input_wom'
OUTPUT = '/Users/woolee/mldl_project/github/crop_faces_from_image/output_wom'
#INPUT = 'input' # directory containing images that have faces
#OUTPUT = 'output' # directory of cropped faces
MODEL = 'face_detector'
CONFIDENCE = 0.5

# load the serialized face detector model
print("[INFO] loading face detector model...")
prototxtPath = os.path.sep.join([MODEL, "deploy.prototxt"])
weightsPath = os.path.sep.join([MODEL, "res10_300x300_ssd_iter_140000.caffemodel"])
net = cv2.dnn.readNet(prototxtPath, weightsPath)

[INFO] loading face detector model...


In [8]:
n = 0
for file in os.listdir(INPUT):
    # load the input image and grab the image spatial dimensions
    image = cv2.imread(f'{INPUT}/{file}')
    
    # wc - error pass
    #while (image is not None):
    #print(file)
    #print(image)
    if (image is None):
        pass
    else:
        (h, w) = image.shape[:2]


        # construct a blob from the image
        blob = cv2.dnn.blobFromImage(image, 1.0, (300, 300), (104.0, 177.0, 123.0))

        # pass the blob through the network and obtain the face detections
        net.setInput(blob)
        detections = net.forward()

        # loop over the detections
        for i in range(0, detections.shape[2]):
            # extract the confidence (i.e., probability) associated with the detection
            confidence = detections[0, 0, i, 2]

            # filter out weak detections by ensuring the confidence is greater than the minimum confidence
            if confidence > CONFIDENCE:
                # compute the (x, y)-coordinates of the bounding box for the object
                box = detections[0, 0, i, 3:7] * np.array([w, h, w, h])
                (startX, startY, endX, endY) = box.astype("int") # pixels of image

                # ensure the bounding boxes fall within the dimensions of the frame
                (startX, startY) = (max(0, startX), max(0, startY))
                (endX, endY) = (min(w - 1, endX), min(h - 1, endY))
                                
                # wc - error pass
                face = Image.open(f'{INPUT}/{file}').crop((startX, startY, endX, endY))
                try:
                    face.save(f'{OUTPUT}/face{n}.jpg')
                except SystemError:
                    pass
                except OSError:
                    pass
    n += 1
